## Handling the Imbalanced Dataset
<p>Consider a credit card fraud detection model where we have 99000 not fraudulent transactions and 1000 fraudulent transactions. So, however dumb the model is, it will give a accuracy of about 90% since the data is totally imbalanced. <p>

## 1. Under sampling majority class
![undersampling](undersampling.png) <br> <br>
But this technique is not effective since we remove lot of data.

## 2. Over sampling minority class by duplication of minority class.
![oversampling](oversampling.png) <br> <br>
Bu this may not be the best technique since we are duplicating the minority class.

## 3. Over sampling minority class class using SMOTE
![smote](smote.png) <br> <br>

## 4. Ensemble Method
![ensemble](ensemble.png) <br> <br>
Build a model with every batch and take majority vote of classification.
<br> 
## 5. Focal Loss
Focal loss will penalize majority samples during los  calculation and give more weight to minority class samples.

In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')


In [47]:
df = pd.read_csv('customer_churn_scaled.csv')
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,InternetService_0,InternetService_DSL,InternetService_Fiber optic,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,0,1,0,0.000000,0,0,0,1,0,...,0,1,0,1,0,0,0,0,1,0
1,1,0,0,0,0.464789,1,0,1,0,1,...,0,1,0,0,1,0,0,0,0,1
2,1,0,0,0,0.014085,1,0,1,1,0,...,0,1,0,1,0,0,0,0,0,1
3,1,0,0,0,0.619718,0,0,1,0,1,...,0,1,0,0,1,0,1,0,0,0
4,0,0,0,0,0.014085,1,0,0,0,0,...,0,0,1,1,0,0,0,0,1,0


In [48]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('Churn', axis=1), df['Churn'], test_size=0.2, random_state=42)


In [49]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5625, 26), (1407, 26), (5625,), (1407,))

In [50]:
def ANN(X_train, y_train, X_test, y_test, loss, weights):
    model = keras.Sequential()
    model.add(keras.layers.Dense(26, input_dim=X_train.shape[1], activation='relu'))
    model.add(keras.layers.Dense(15, activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    model.compile(loss=loss, optimizer='adam', metrics=['accuracy'])

    if weights == -1:
        model.fit(X_train, y_train, epochs=100)
    else:
        model.fit(X_train, y_train, epochs=100, class_weight=weights)

    print(model.evaluate(X_test, y_test))


    y_pred = model.predict(X_test)
    y_pred = np.round(y_pred)

    print('Classification Report: \n', classification_report(y_test, y_pred))

    return y_pred

In [51]:
y_pred = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
176/176 [==============================] - 1s 1ms/step - loss: 0.5223 - accuracy: 0.7428
Epoch 2/100
176/176 [==============================] - 0s 1ms/step - loss: 0.4260 - accuracy: 0.7972
Epoch 3/100
176/176 [==============================] - 0s 1ms/step - loss: 0.4210 - accuracy: 0.7977
Epoch 4/100
176/176 [==============================] - 0s 1ms/step - loss: 0.4147 - accuracy: 0.8048
Epoch 5/100
176/176 [==============================] - 0s 1ms/step - loss: 0.4120 - accuracy: 0.8041
Epoch 6/100
176/176 [==============================] - 0s 1ms/step - loss: 0.4107 - accuracy: 0.8096
Epoch 7/100
176/176 [==============================] - 0s 1ms/step - loss: 0.4097 - accuracy: 0.8075
Epoch 8/100
176/176 [==============================] - 0s 1ms/step - loss: 0.4077 - accuracy: 0.8098
Epoch 9/100
176/176 [==============================] - 0s 1ms/step - loss: 0.4063 - accuracy: 0.8110
Epoch 10/100
176/176 [==============================] - 0s 2ms/step - loss: 0.4044 - accura

In [52]:
y_test.value_counts()

0    1033
1     374
Name: Churn, dtype: int64

As we can see the data is partially imbalanced. So, let us perform the above discussed techniques to handle the imbalanced dataset.

In [53]:
count_class_0, count_class_1 = df.Churn.value_counts()

df_class_0 = df[df['Churn'] == 0]
df_class_1 = df[df['Churn'] == 1]


In [54]:
df_class_0.shape, df_class_1.shape

((5163, 27), (1869, 27))

Data imbalance is seen here.

## 1. Under Sampling

In [55]:
df_class_0_under = df_class_0.sample(count_class_1)
df_class_0_under.shape

(1869, 27)

In [56]:
df_class_0_under.shape, df_class_1.shape

((1869, 27), (1869, 27))

Now, we got the sample number of data records. Let's concatenate them.

In [57]:
df_test_under = pd.concat([df_class_0_under, df_class_1])
df_test_under.shape

(3738, 27)

In [58]:
X_train, X_test, y_train, y_test = train_test_split(df_test_under.drop('Churn', axis=1), df_test_under['Churn'], test_size=0.2, random_state=15, stratify=df_test_under['Churn'])
# stratify will ensure that X_train and X_test have the same proportions of 0 and 1

In [59]:
y_train.value_counts()

1    1495
0    1495
Name: Churn, dtype: int64

In [60]:
y_pred = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
94/94 [==============================] - 1s 1ms/step - loss: 0.6312 - accuracy: 0.6428
Epoch 2/100
94/94 [==============================] - 0s 1ms/step - loss: 0.5178 - accuracy: 0.7492
Epoch 3/100
94/94 [==============================] - 0s 1ms/step - loss: 0.5007 - accuracy: 0.7579
Epoch 4/100
94/94 [==============================] - 0s 1ms/step - loss: 0.4933 - accuracy: 0.7642
Epoch 5/100
94/94 [==============================] - 0s 1ms/step - loss: 0.4877 - accuracy: 0.7672
Epoch 6/100
94/94 [==============================] - 0s 1ms/step - loss: 0.4848 - accuracy: 0.7686
Epoch 7/100
94/94 [==============================] - 0s 1ms/step - loss: 0.4831 - accuracy: 0.7689
Epoch 8/100
94/94 [==============================] - 0s 1ms/step - loss: 0.4794 - accuracy: 0.7712
Epoch 9/100
94/94 [==============================] - 0s 1ms/step - loss: 0.4766 - accuracy: 0.7716
Epoch 10/100
94/94 [==============================] - 0s 1ms/step - loss: 0.4745 - accuracy: 0.7716
Epoch 11/

We can see that precision and recall has improved because of undersampling.

## 2. Over Sampling

In [61]:
count_class_0, count_class_1

(5163, 1869)

In [62]:
df_class_1_over = df_class_1.sample(count_class_0, replace=True) # duplication

In [63]:
df_class_0.shape, df_class_1_over.shape

((5163, 27), (5163, 27))

In [64]:
df_test_over = pd.concat([df_class_0, df_class_1_over])
df_test_over.shape

(10326, 27)

In [65]:
df_test_over['Churn'].value_counts()

1    5163
0    5163
Name: Churn, dtype: int64

In [66]:
X_train, X_test, y_train, y_test = train_test_split(df_test_over.drop('Churn', axis=1), df_test_over['Churn'], test_size=0.2, random_state=15, stratify=df_test_over['Churn'])

In [67]:
y_train.value_counts() # the data is equally split since stratify=df_test_over['Churn']

1    4130
0    4130
Name: Churn, dtype: int64

In [68]:
y_pred = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
259/259 [==============================] - 1s 861us/step - loss: 0.5444 - accuracy: 0.7242
Epoch 2/100
259/259 [==============================] - 0s 841us/step - loss: 0.4929 - accuracy: 0.7587
Epoch 3/100
259/259 [==============================] - 0s 868us/step - loss: 0.4865 - accuracy: 0.7638
Epoch 4/100
259/259 [==============================] - 0s 861us/step - loss: 0.4820 - accuracy: 0.7677
Epoch 5/100
259/259 [==============================] - 0s 853us/step - loss: 0.4769 - accuracy: 0.7702
Epoch 6/100
259/259 [==============================] - 0s 864us/step - loss: 0.4730 - accuracy: 0.7700
Epoch 7/100
259/259 [==============================] - 0s 1ms/step - loss: 0.4699 - accuracy: 0.7754
Epoch 8/100
259/259 [==============================] - 0s 1ms/step - loss: 0.4666 - accuracy: 0.7759
Epoch 9/100
259/259 [==============================] - 0s 1ms/step - loss: 0.4632 - accuracy: 0.7759
Epoch 10/100
259/259 [==============================] - 0s 1ms/step - loss: 0.4

Oversampling performs even better.

## 3. SMOTE

In [69]:
!pip install imbalanced-learn

In [70]:
X = df.drop('Churn', axis='columns')
y = df['Churn']

In [71]:
y.value_counts()

0    5163
1    1869
Name: Churn, dtype: int64

In [72]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
X_res, y_res = smote.fit_resample(X, y)

y_res.value_counts()


1    5163
0    5163
Name: Churn, dtype: int64

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42, stratify=y_res)

In [74]:
y_pred = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
259/259 [==============================] - 1s 915us/step - loss: 0.5385 - accuracy: 0.7443
Epoch 2/100
259/259 [==============================] - 0s 1ms/step - loss: 0.4683 - accuracy: 0.7809
Epoch 3/100
259/259 [==============================] - 0s 2ms/step - loss: 0.4582 - accuracy: 0.7824
Epoch 4/100
259/259 [==============================] - 0s 1ms/step - loss: 0.4517 - accuracy: 0.7844
Epoch 5/100
259/259 [==============================] - 0s 1ms/step - loss: 0.4459 - accuracy: 0.7883
Epoch 6/100
259/259 [==============================] - 0s 2ms/step - loss: 0.4402 - accuracy: 0.7931
Epoch 7/100
259/259 [==============================] - 0s 1ms/step - loss: 0.4350 - accuracy: 0.7965
Epoch 8/100
259/259 [==============================] - 0s 1ms/step - loss: 0.4307 - accuracy: 0.7977
Epoch 9/100
259/259 [==============================] - 0s 1ms/step - loss: 0.4280 - accuracy: 0.7966
Epoch 10/100
259/259 [==============================] - 0s 1ms/step - loss: 0.4233 - accu

Recall, Precision, Accuracy everything has been improved very well.

## 4. Use of Ensemble with undersampling

In [75]:
df.Churn.value_counts()

0    5163
1    1869
Name: Churn, dtype: int64

In [77]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('Churn', axis=1), df['Churn'], test_size=0.2, random_state=43, stratify=df['Churn'])

In [81]:
y_train.value_counts()

0    4130
1    1495
Name: Churn, dtype: int64

In [82]:
df3 = X_train.copy()
df3['Churn'] = y_train

In [83]:
df3_class0 = df3[df3.Churn == 0]
df3_class1 = df3[df3.Churn == 1]

In [84]:
4130/1495

2.762541806020067

Divide the majority dataset into 3 parts.

In [85]:
def get_train_batch(df_majority, df_minority, start, end):
    df_train = pd.concat([df_majority[start:end], df_minority], axis=0)

    X_train = df_train.drop('Churn', axis='columns')
    y_train = df_train.Churn

    return X_train, y_train

### Batch 1

In [91]:
X_train, y_train = get_train_batch(df3_class0, df3_class1, 0, 1495)
X_train.shape

(2990, 26)

In [93]:
y_pred1 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)


Epoch 1/100
94/94 [==============================] - 1s 1ms/step - loss: 0.6231 - accuracy: 0.6672
Epoch 2/100
94/94 [==============================] - 0s 1ms/step - loss: 0.5017 - accuracy: 0.7579
Epoch 3/100
94/94 [==============================] - 0s 1ms/step - loss: 0.4866 - accuracy: 0.7669
Epoch 4/100
94/94 [==============================] - 0s 1ms/step - loss: 0.4796 - accuracy: 0.7652
Epoch 5/100
94/94 [==============================] - 0s 1ms/step - loss: 0.4766 - accuracy: 0.7729
Epoch 6/100
94/94 [==============================] - 0s 1ms/step - loss: 0.4745 - accuracy: 0.7702
Epoch 7/100
94/94 [==============================] - 0s 1ms/step - loss: 0.4714 - accuracy: 0.7736
Epoch 8/100
94/94 [==============================] - 0s 1ms/step - loss: 0.4709 - accuracy: 0.7719
Epoch 9/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4671 - accuracy: 0.7769
Epoch 10/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4648 - accuracy: 0.7799
Epoch 11/

## Batch 2

In [95]:
X_train, y_train = get_train_batch(df3_class0, df3_class1, 1495, 1495*2)

y_pred2 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)


Epoch 1/100
94/94 [==============================] - 1s 2ms/step - loss: 0.6241 - accuracy: 0.6532
Epoch 2/100
94/94 [==============================] - 0s 2ms/step - loss: 0.5346 - accuracy: 0.7355
Epoch 3/100
94/94 [==============================] - 0s 2ms/step - loss: 0.5201 - accuracy: 0.7428
Epoch 4/100
94/94 [==============================] - 0s 1ms/step - loss: 0.5118 - accuracy: 0.7492
Epoch 5/100
94/94 [==============================] - 0s 1ms/step - loss: 0.5065 - accuracy: 0.7542
Epoch 6/100
94/94 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.7518
Epoch 7/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4983 - accuracy: 0.7538
Epoch 8/100
94/94 [==============================] - 0s 1ms/step - loss: 0.4973 - accuracy: 0.7555
Epoch 9/100
94/94 [==============================] - 0s 1ms/step - loss: 0.4933 - accuracy: 0.7545
Epoch 10/100
94/94 [==============================] - 0s 1ms/step - loss: 0.4910 - accuracy: 0.7582
Epoch 11/

## Batch 3

In [97]:
X_train, y_train = get_train_batch(df3_class0, df3_class1, 2990, 4130)

y_pred3 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)


Epoch 1/100
83/83 [==============================] - 1s 3ms/step - loss: 0.5865 - accuracy: 0.7366
Epoch 2/100
83/83 [==============================] - 0s 3ms/step - loss: 0.5057 - accuracy: 0.7670
Epoch 3/100
83/83 [==============================] - 0s 2ms/step - loss: 0.4902 - accuracy: 0.7727
Epoch 4/100
83/83 [==============================] - 0s 2ms/step - loss: 0.4830 - accuracy: 0.7708
Epoch 5/100
83/83 [==============================] - 0s 2ms/step - loss: 0.4799 - accuracy: 0.7734
Epoch 6/100
83/83 [==============================] - 0s 2ms/step - loss: 0.4764 - accuracy: 0.7750
Epoch 7/100
83/83 [==============================] - 0s 2ms/step - loss: 0.4741 - accuracy: 0.7784
Epoch 8/100
83/83 [==============================] - 0s 2ms/step - loss: 0.4728 - accuracy: 0.7780
Epoch 9/100
83/83 [==============================] - 0s 2ms/step - loss: 0.4701 - accuracy: 0.7791
Epoch 10/100
83/83 [==============================] - 0s 2ms/step - loss: 0.4682 - accuracy: 0.7829
Epoch 11/

Majority wins...

In [98]:
y_pred_final = y_pred1.copy()

for i in range(len(y_pred1)):
    n_ones = y_pred1[i] + y_pred2[i] + y_pred3[i]
    if n_ones > 1:
        y_pred_final[i] = 1
    else:
        y_pred_final[i] = 0

In [99]:
print(classification_report(y_test, y_pred_final))

              precision    recall  f1-score   support

           0       0.92      0.66      0.77      1033
           1       0.47      0.83      0.60       374

    accuracy                           0.71      1407
   macro avg       0.69      0.75      0.68      1407
weighted avg       0.80      0.71      0.72      1407

